#  Pipeline de Limpeza e Preparação de Dados - ANAC

Este script realiza o pré-processamento dos dados históricos de voos da ANAC (2023-2025), transformando os dados brutos em um dataset pronto para treinamento de modelos de Machine Learning.

## Visão Geral do Processo

O pipeline executa as seguintes etapas principais:
1. **Ingestão de Dados:** Carregamento de grandes volumes de dados brutos (CSV).
2. **Seleção de Features:** Filtragem apenas das colunas relevantes para a previsão.
3. **Enriquecimento (ICAO -> IATA):** Conversão de códigos de aeroportos para o padrão comercial.
4. **Engenharia de Atributos (Target):** Criação da variável alvo binária (Atraso vs. Pontual).
5. **Limpeza Final:** Remoção de vazamento de dados (data leakage) e voos cancelados.

##Importações de bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
from google.colab import files

##Montando o drive e carregando os arquivos


O script utiliza `pandas` para manipulação de dados e conecta ao Google Drive para acessar os arquivos brutos.
- **Verificação de Segurança:** Antes de carregar, o script valida se o caminho do arquivo existe para evitar quebras de execução.
- **Tratamento de Erros:** Utiliza `on_bad_lines='skip'` para ignorar linhas mal formatadas no arquivo original da ANAC.

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2. Caminho do Arquivo
caminho_arquivo = "/content/drive/MyDrive/Dados_ANAC/dados_anac_completo_2023_2025.csv"

# Verificação de segurança: O arquivo existe?
if not os.path.exists(caminho_arquivo):
    print(f"ERRO: Arquivo não encontrado em: {caminho_arquivo}")
else:
    print(f"Arquivo encontrado! Tamanho: {os.path.getsize(caminho_arquivo) / (1024*1024):.2f} MB")

    # 3. Carregar o DataFrame
    print("Carregando dados...")
    df = pd.read_csv(caminho_arquivo, sep=';', on_bad_lines='skip', engine='python')

Arquivo encontrado! Tamanho: 824.11 MB
Carregando dados...


### Seleção de Colunas
Para reduzir a dimensionalidade e focar nas variáveis preditivas, selecionamos inicialmente:
- `sg_empresa_icao`: Sigla da companhia aérea.
- `sg_icao_origem/destino`: Aeroportos (posteriormente convertidos).
- `dt_partida_prevista`: Data/hora planejada (essencial para extrair sazonalidade).
- `ds_situacao_voo/chegada`: Usados apenas para criar o target e depois removidos.

In [ ]:
colunas_selecionadas = [
    'sg_empresa_icao',
    'sg_icao_origem',
    'sg_icao_destino',
    'sg_equipamento_icao',
    'nr_assentos_ofertados',
    'cd_tipo_linha',
    'nr_voo',
    'dt_partida_prevista',
    'ds_situacao_voo',
    'ds_situacao_chegada',
]

In [ ]:
df_limpo = df[colunas_selecionadas].copy()
df_limpo.info()
df_limpo.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3048182 entries, 0 to 3048181
Data columns (total 10 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   sg_empresa_icao        object
 1   sg_icao_origem         object
 2   sg_icao_destino        object
 3   sg_equipamento_icao    object
 4   nr_assentos_ofertados  int64 
 5   cd_tipo_linha          object
 6   nr_voo                 object
 7   dt_partida_prevista    object
 8   ds_situacao_voo        object
 9   ds_situacao_chegada    object
dtypes: int64(1), object(9)
memory usage: 232.6+ MB


,sg_empresa_icao,sg_icao_origem,sg_icao_destino,sg_equipamento_icao,nr_assentos_ofertados,cd_tipo_linha,nr_voo,dt_partida_prevista,ds_situacao_voo,ds_situacao_chegada
0,AAL,SBGL,KMIA,B788,295,I,0904,01/01/2023 00:05,REALIZADO,Antecipado
1,AAL,KMIA,SBGL,B788,295,I,0905,01/01/2023 21:45,REALIZADO,Antecipado
2,AAL,SBGR,KMIA,B77W,318,I,0906,01/01/2023 00:00,REALIZADO,Atraso 60-120
3,AAL,KMIA,SBGR,B77W,318,I,0929,01/01/2023 21:45,REALIZADO,Pontual
4,AAL,SBGR,KMIA,B772,288,I,0930,01/01/2023 22:00,REALIZADO,Antecipado


##Carregamento do mapa de aeroportos
Dados encontados no link: https://www.worlddata.info/downloads/airports.csv

In [ ]:
airports_ref = pd.read_csv('airports.csv')
airports_map = airports_ref[['ICAO', 'IATA']].dropna(subset=['ICAO'])

### Conversão de ICAO para IATA
A maioria dos usuários finais reconhece aeroportos pelo código IATA (ex: GRU, JFK) e não ICAO (ex: SBGR, KJFK).
- **Processo:** Realiza um `merge` (join) com um dataset auxiliar (`airports.csv`).
- **Lógica:** Mapeia a origem e o destino para IATA e descarta as colunas ICAO originais.

In [ ]:
# Converter a ORIGEM (sg_icao_origem):
df_resultado = df_limpo.merge(airports_map, left_on='sg_icao_origem', right_on='ICAO', how='left')
df_resultado = df_resultado.rename(columns={'IATA': 'sg_iata_origem'}).drop(columns=['ICAO'])

# Converter o DESTINO (sg_icao_destino):
df_resultado = df_resultado.merge(airports_map, left_on='sg_icao_destino', right_on='ICAO', how='left')
df_resultado = df_resultado.rename(columns={'IATA': 'sg_iata_destino'}).drop(columns=['ICAO'])

### Criação da Variável Alvo (Target)
Esta é a etapa crítica para o aprendizado supervisionado. A função `classificar_target` define o que o modelo deve aprender:

| Situação do Voo | Classificação | Valor do Target | Ação |
| :--- | :--- | :--- | :--- |
| **Atraso** | Classe Positiva | `1` | Mantido |
| **Pontual / Antecipado** | Classe Negativa | `0` | Mantido |
| **Cancelado** | Outliers/Ruído | `-1` | **Removido** |

In [ ]:
colunas_selecionadas = [
    'sg_empresa_icao',
    'sg_iata_origem',
    'sg_iata_destino',
    'sg_equipamento_icao',
    'nr_assentos_ofertados',
    'cd_tipo_linha',
    'nr_voo',
    'dt_partida_prevista',
    'ds_situacao_voo',
    'ds_situacao_chegada',
]
df_limpo = df_resultado[colunas_selecionadas].copy()

In [ ]:
df_limpo.info()
df_limpo.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3048182 entries, 0 to 3048181
Data columns (total 10 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   sg_empresa_icao        object
 1   sg_iata_origem         object
 2   sg_iata_destino        object
 3   sg_equipamento_icao    object
 4   nr_assentos_ofertados  int64 
 5   cd_tipo_linha          object
 6   nr_voo                 object
 7   dt_partida_prevista    object
 8   ds_situacao_voo        object
 9   ds_situacao_chegada    object
dtypes: int64(1), object(9)
memory usage: 232.6+ MB


,sg_empresa_icao,sg_iata_origem,sg_iata_destino,sg_equipamento_icao,nr_assentos_ofertados,cd_tipo_linha,nr_voo,dt_partida_prevista,ds_situacao_voo,ds_situacao_chegada
0,AAL,GIG,MIA,B788,295,I,0904,01/01/2023 00:05,REALIZADO,Antecipado
1,AAL,MIA,GIG,B788,295,I,0905,01/01/2023 21:45,REALIZADO,Antecipado
2,AAL,GRU,MIA,B77W,318,I,0906,01/01/2023 00:00,REALIZADO,Atraso 60-120
3,AAL,MIA,GRU,B77W,318,I,0929,01/01/2023 21:45,REALIZADO,Pontual
4,AAL,GRU,MIA,B772,288,I,0930,01/01/2023 22:00,REALIZADO,Antecipado


In [ ]:
def classificar_target(row):
    status_chegada = str(row['ds_situacao_chegada'])
    status_voo = str(row['ds_situacao_voo'])

    # 1. CANCELADOS -> TARGET 1
    if status_voo == 'CANCELADO':
        return -1

    # 2. ATRASOS -> TARGET 1
    if 'Atraso' in status_chegada:
        return 1

    # 3. PONTUAL / ANTECIPADO -> TARGET 0
    if 'Pontual' in status_chegada or 'Antecipado' in status_chegada:
        return 0

    # 4. RESTO -> DESCARTAR (-1)
    return -1
df_limpo['target'] = df_limpo.apply(classificar_target, axis=1)
df_limpo.head()

,sg_empresa_icao,sg_iata_origem,sg_iata_destino,sg_equipamento_icao,nr_assentos_ofertados,cd_tipo_linha,nr_voo,dt_partida_prevista,ds_situacao_voo,ds_situacao_chegada,target
0,AAL,GIG,MIA,B788,295,I,0904,01/01/2023 00:05,REALIZADO,Antecipado,0
1,AAL,MIA,GIG,B788,295,I,0905,01/01/2023 21:45,REALIZADO,Antecipado,0
2,AAL,GRU,MIA,B77W,318,I,0906,01/01/2023 00:00,REALIZADO,Atraso 60-120,1
3,AAL,MIA,GRU,B77W,318,I,0929,01/01/2023 21:45,REALIZADO,Pontual,0
4,AAL,GRU,MIA,B772,288,I,0930,01/01/2023 22:00,REALIZADO,Antecipado,0


### Limpeza Final e Prevenção de Data Leakage
Antes de exportar:
1. **Remoção de linhas inválidas:** Filtra todos os registros onde `target == -1`.
2. **Remoção de Colunas "Spoiler":** As colunas `ds_situacao_chegada` e `ds_situacao_voo` são removidas.
   - *Motivo:* Elas contêm a resposta do problema. Se mantidas, o modelo teria 100% de acurácia artificialmente (vazamento de dados).

In [ ]:
df_limpo = df_limpo[df_limpo['target'] != -1].copy()
cols_remover = ['ds_situacao_chegada', 'ds_situacao_voo']
df_limpo.drop(columns=cols_remover, inplace=True)

In [ ]:
df_limpo.info()
df_limpo.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2820120 entries, 0 to 3048138
Data columns (total 9 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   sg_empresa_icao        object
 1   sg_iata_origem         object
 2   sg_iata_destino        object
 3   sg_equipamento_icao    object
 4   nr_assentos_ofertados  int64 
 5   cd_tipo_linha          object
 6   nr_voo                 object
 7   dt_partida_prevista    object
 8   target                 int64 
dtypes: int64(2), object(7)
memory usage: 215.2+ MB


,sg_empresa_icao,sg_iata_origem,sg_iata_destino,sg_equipamento_icao,nr_assentos_ofertados,cd_tipo_linha,nr_voo,dt_partida_prevista,target
0,AAL,GIG,MIA,B788,295,I,0904,01/01/2023 00:05,0
1,AAL,MIA,GIG,B788,295,I,0905,01/01/2023 21:45,0
2,AAL,GRU,MIA,B77W,318,I,0906,01/01/2023 00:00,1
3,AAL,MIA,GRU,B77W,318,I,0929,01/01/2023 21:45,0
4,AAL,GRU,MIA,B772,288,I,0930,01/01/2023 22:00,0


###Exportação
O dataset limpo é salvo como `dados_limpos_anac.csv` com codificação `utf-8-sig` para garantir compatibilidade com caracteres especiais e Excel.


In [ ]:
caminho_saida = "/content/drive/MyDrive/Dados_ANAC/dados_limpos_anac.csv"
df_limpo.to_csv(caminho_saida, index=False, sep=';', encoding='utf-8-sig')

print(f"DataFrame salvo com sucesso em: {caminho_saida}")

DataFrame salvo com sucesso em: /content/drive/MyDrive/Dados_ANAC/dados_limpos_anac.csv
